Bounding f_n(•) and h_n(•) with Mixed Integer Programming

In [1]:
%run constants.ipynb
%run gen_h.ipynb

In [2]:
import gurobipy as gp
from gurobipy import GRB
from itertools import repeat
import re

In [3]:
def var_name(*ineqs, prefix=''):
    '''
    Returns the variable name for the MIP.
    '''
    return prefix + f'{str(ineqs)}'

In [4]:
def extract_ineq(var):
    '''
    Extracts the inequality from the given variable's name.
    (For interpretability, we associate each indicator variable's name 
    with the inequality(s) they represent)
    '''
    match = re.findall(r'\(([^()]+)\)', var.varName)
    return sympify(match[0]) if match else None

In [5]:
def extract_ineqs(variables):
    '''
    Extracts inequalities from multiple variables, 
    based on their values (i.e. 1 = true, 0 = false).
    '''
    ineqs = []
    for v in variables:
        e = extract_ineq(v)
        if v.x == 1 and e is not None:  # indicator is true
            ineqs.append(e)
        elif v.x == 0:  # indicator is false
            ineqs.append(Not(And(e)))
    return ineqs

In [30]:
def to_gp_linexpr(sp_expr, var_map):
    '''
    Converts a sympy expression to a gurobipy linear expression,
    using the provided (sp : gp) variable mapping.
    '''
    gp_expr = gp.LinExpr()

    if isinstance(sp_expr, Add):
        for term in sp_expr.args:
            gp_expr.add(to_gp_linexpr(term, var_map))
    elif sp_expr.is_Mul:
        coeff = 1
        var = None
        for factor in sp_expr.args:
            if factor.is_number:  # sp coefficient
                coeff *= float(factor)
            elif factor in var_map:  # sp variable
                var = var_map[factor]
        if var is not None:
            gp_expr.addTerms(coeff, var)
        else:  # constant
            gp_expr.addConstant(coeff)
    elif sp_expr in var_map:  # single variable expression
        gp_expr.addTerms(1.0, var_map[sp_expr])
    elif sp_expr.is_number:   # single constant
        gp_expr.addConstant(float(sp_expr))

    return gp_expr

In [7]:
EPS = 1
def unify_ineq(ineq):
    '''
    Converts the given inequality to the form expr >= 0.
    '''
    lhs, rhs, op = ineq.lhs, ineq.rhs, ineq.rel_op
    # Check the inequality type and unify the format
    if op == '>=':
        # lhs >= rhs -> lhs - rhs >= 0
        return Rel(lhs - rhs, 0, '>=')
    elif op == '>':
        # lhs > rhs -> lhs - rhs > 0 => lhs - rhs >= EPS
        return Rel(lhs - rhs - EPS, 0, '>=')
    elif op == '<=':
        # lhs <= rhs -> lhs - rhs <= 0 -> rhs - lhs >= 0
        return Rel(rhs - lhs, 0, '>=')
    elif op == '<':
        # lhs < rhs -> rhs > lhs -> rhs - lhs > 0 -> rhs - lhs >= EPS
        return Rel(rhs - lhs - EPS, 0, '>=')
    else:
        raise ValueError(f"Unhandled inequality type: {op}")

In [17]:
def to_gp_ineq(ineq, var_map):
    '''
    Like `to_gp_linexpr`, but converts a sympy inequality to a gurobipy inequality 
    (of the form >= 0).
    '''
    return (to_gp_linexpr(unify_ineq(ineq).lhs, var_map) >= 0)

In [18]:
def add_ind_constr(ineq, model=None, name='ind', var_map=None):
    ''' 
    Adds indicator constraint to the model for a single inequality. 
    '''
    r = unify_ineq(ineq)  # to the form >= 0
    lhs = to_gp_linexpr(r.lhs, var_map)
    v = model.addVar(vtype=GRB.BINARY, name=name)
    model.addConstr((v == 1) >> (lhs >= 0))
    model.addConstr((v == 0) >> (lhs <= -EPS))  # lhs < 0
    return v

In [19]:
def add_ind_constrs(*ineqs, model=None, var_map=None):
    ''' 
    Adds variables and constraints to the model for a compound indicator 
    (i.e. may represent the and of multiple inequalities). 
    '''
    if len(ineqs) == 1:
        return (add_ind_constr(ineqs[0], 
                               name=var_name(ineqs[0], prefix='single_ind'), 
                               var_map=var_map,
                               model=model),)
    
    I = model.addVar(vtype=GRB.BINARY, 
                     name=var_name(*ineqs, prefix='main_ind'))
    component_vars = [add_ind_constr(i, 
                                     name=var_name(i, prefix='component_ind'), 
                                     var_map=var_map,
                                     model=model) for i in ineqs]
    model.addConstr(I == gp.and_(component_vars), 
                    name=var_name(*ineqs, prefix='andconstr'))
    return I, component_vars

In [42]:
def analyse_result(model, relax=False):
    '''
    Analyses the status of the model.
    If the model is feasible and `relax` is True, then re-optimizes the model with relaxed bounds.
    Note that if the relaxation is infeasible, then the exact model must too be infeasible
    '''
    if model.status == GRB.OPTIMAL:
        print('Optimal solution found:')
        for v in model.getVars():
            print(f"{v.varName}: {v.x}")
        print(f'Objective value:', model.objVal)
        print(f'MIPGap = |ObjBound - ObjVal|/|ObjVal|:', model.MIPGap)
    elif model.status == GRB.UNBOUNDED:
        print('The model cannot be solved because it is unbounded')
    elif model.status != GRB.INF_OR_UNBD and model.status != GRB.INFEASIBLE:
        print('Optimization was stopped with status %d' % model.status)
    else:
        # Relax the constraints and try to make the model feasible
        print('The model is infeasible')
        if relax:
            print('Relaxing the bounds')
            model.feasRelaxS(1, False, False, True)
            model.optimize()
            analyse_result(model, relax=relax)

In [50]:
def f_mip(n, output_file=False, analyse=True, relax=False):
    '''
    Runs MIP model to minimize f_n - thresh(n).
    If `output_file` is True, writes the model as a '.lp' file.
    If `analyse` is True, analyses the optimization results.
    '''
    # uncomment the following to collect the regions cut out (i.e. excluded) by the model
#     failed_regions = []
    
#     def extract_region(model):
#         region = []
#         variables = model.getVars()
#         sol = model.cbGetSolution(variables)
#         for v, val in zip(variables, sol):
#             # only look at the individual indicators
#             if not 'single_ind' in v.varName and not 'component_ind' in v.varName:
#                 continue
#             e = extract_ineq(v)
#             if val == 1 and e is not None:
#                 region.append(e[0])
#             elif val == 0:
#                 # indicator is false
#                 # guaranteed single inequality
#                 region.append(Not(e[0]))
#         return region
    
#     def pos_f_callback(model, where):
#         if where == GRB.Callback.MIPSOL:
#             # Capture node information
#             try:
#                 obj = model.cbGet(GRB.Callback.MIPSOL_OBJ)
#                 print("Current objective:", obj)
#                 if obj > 0:  # Node is infeasible
#                     failed_regions.append(extract_region(model))
#             except gp.GurobiError as e:
#                 print(f"Gurobi error during callback: {e}")
#             except Exception as e:
#                 print(f"Unexpected error during callback: {e}")
    
    h_xyz = read(H_XYZ_CACHE, min_n=1, max_n=n)
    h_yxz = read(H_YXZ_CACHE, min_n=1, max_n=n)
    num_pos_per_n = [len(H_XYZ[i][1]) for i in range(1,n+1)]
    num_neg_per_n = [len(H_YXZ[i][1]) for i in range(1,n+1)]
    num_pos = sum(num_pos_per_n)
    num_neg = sum(num_neg_per_n)
    
    pos_coefs = concat([repeat(j, k) for j,k in zip([coef(i) for i in range(1,n+1)], num_pos_per_n)])
    neg_coefs = concat([repeat(j, k) for j,k in zip([-coef(i) for i in range(1,n+1)], num_neg_per_n)])
    const = sum([coef(i)*(H_XYZ[i][0] - H_YXZ[i][0]) for i in range(1,n+1)])
    
    all_pos_comp_inds = concat([H_XYZ[i][1] for i in range(1,n+1)])
    all_neg_comp_inds = concat([H_YXZ[i][1] for i in range(1,n+1)])
    
    model = gp.Model(f'f_{n}(x,y,z)-thresh{n}', env=gp.Env())
    model.Params.OutputFlag = 1
    model.Params.LogToConsole = 1
    model.Params.MIPFocus = 1
    
    gx = model.addVar(lb=1, vtype='I', name='x')
    gy = model.addVar(lb=1, vtype='I', name='y')
    gz = model.addVar(lb=1, vtype='I', name='z')
    sp_to_gp = {x: gx, y: gy, z: gz}
    
    pos_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp) \
                for i in all_pos_comp_inds]
    neg_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp) \
                for i in all_neg_comp_inds]
    
    obj = gp.quicksum([pos_coefs[i]*pos_inds[i][0] for i in range(num_pos)]) + \
        gp.quicksum([neg_coefs[i]*neg_inds[i][0] for i in range(num_neg)]) + const - thresh(n)
    
    model.setObjective(obj, GRB.MINIMIZE)
    f_constr = model.addConstr(obj <= 0, name='non_pos_f')
    # constraints for V
    for i, ineq in enumerate(V):
        model.addConstr(to_gp_ineq(ineq, sp_to_gp), name=f'V{i}')
    
#     model.optimize(pos_f_callback)
    model.optimize()
    
    if analyse:
        analyse_result(model, relax=relax)
    
    if output_file:
        model.write(f'f_{n}(x,y,z)-thresh{n}.lp')
    
    return ((model.status == GRB.OPTIMAL and \
            model.MIPGap == 0 and \
            eval_f(n, gx.x, gy.x, gz.x) < 0), # sanity check using the x,y,z values from the model
            model)

In [51]:
f_mip(4, analyse=True, relax=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Set parameter MIPFocus to value 1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 898 columns and 360 nonzeros
Model fingerprint: 0xedd93d96
Model has 1543 general constraints
Variable types: 0 continuous, 898 integer (895 binary)
Coefficient statistics:
  Matrix range     [8e-04, 1e+00]
  Objective range  [8e-04, 3e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 1e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 2e+01]
Presolve added 2344 rows and 1944 columns
Presolve time: 0.02s
Presolved: 2348 rows, 2842 columns, 7222 nonzeros
Presolved model has 1296 SOS constraint(s)
Variable types: 0 continuous, 2842 integer (1543 binary)

Root relaxation: objective -2.375000e-01, 1327 iterations, 0.01 se

 660493 52557    0.00020  363   20    0.00029    0.00000   100%   9.8  275s
 677021 53974    0.00000 1299   66    0.00029    0.00000   100%   9.7  280s
 690040 56081 infeasible  558         0.00029    0.00000   100%   9.6  285s
 707316 57444    0.00010  768   28    0.00029    0.00000   100%   9.5  290s
 722244 58033    0.00000  941   79    0.00029    0.00000   100%   9.4  295s
 735393 60870    0.00007  613   17    0.00029    0.00000   100%   9.4  300s
 755272 62196    0.00000  577   76    0.00029    0.00000   100%   9.3  305s
 766669 62915 infeasible  393         0.00029    0.00000   100%   9.2  310s
 781107 64146    0.00027 1013   44    0.00029    0.00000   100%   9.2  315s
 795323 65617    0.00010  851   19    0.00029    0.00000   100%   9.1  320s
 810379 67954    0.00000 1335   71    0.00029    0.00000   100%   9.0  325s
 824345 69041    0.00000  641   26    0.00029    0.00000   100%   9.0  330s
 840073 69993 infeasible 1093         0.00029    0.00000   100%   8.9  335s
 852507 7131

# LB on h_n(x,y,z) - h_n(y,x,z)

In [25]:
def h_mip(n, output_file=False, print_result=True):
    '''
    Like `f_mip`, but runs MIP model to minimize h_n(x,y,z) - h_n(y,x,z).
    '''
    h_xyz = read(H_XYZ_CACHE, min_n=1, max_n=n)
    h_yxz = read(H_YXZ_CACHE, min_n=1, max_n=n)
    
    all_pos_comp_inds = concat([H_XYZ[i][1] for i in range(1,n+1)])
    all_neg_comp_inds = concat([H_YXZ[i][1] for i in range(1,n+1)])
    
    model = gp.Model(f'h_{n}(x,y,z)-h_{n}(y,x,z)', env=gp.Env())
    model.Params.OutputFlag = 1
    model.Params.LogToConsole = 1
    model.Params.MIPFocus = 1
    
    gx = model.addVar(lb=1, vtype='I', name='x')
    gy = model.addVar(lb=1, vtype='I', name='y')
    gz = model.addVar(lb=1, vtype='I', name='z')
    sp_to_gp = {x: gx, y: gy, z: gz}
    
    pos_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp) for i in H_XYZ[n][1]]
    neg_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp) for i in H_YXZ[n][1]]
    const = H_XYZ[n][0] - H_YXZ[n][0]
    
    obj = coef(n) * (gp.quicksum([i[0] for i in pos_inds]) - gp.quicksum([i[0] for i in neg_inds]) + const)
    
    model.setObjective(obj, GRB.MINIMIZE)
    # constraints for V
#     model.addConstr(gz <= gx - 1, name='V1')
#     model.addConstr(gx <= gy - 1, name='V2')
    for i, ineq in enumerate(V):
        model.addConstr(to_gp_ineq(ineq, sp_to_gp), name=f'V{i}')
    
    model.optimize()
    
    if print_result:
        print(analyse_result(model))
    
    if output_file:
        model.write(f'h_{n}(x,y,z)-h_{n}(y,x,z).lp')
    
    return (model.status == GRB.OPTIMAL and model.MIPGap == 0), model

In [27]:
h_mip(4, print_result=False)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Set parameter MIPFocus to value 1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 785 columns and 5 nonzeros
Model fingerprint: 0x6e4bd36c
Model has 1342 general constraints
Variable types: 0 continuous, 785 integer (782 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-04, 8e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 2e+01]
Presolve added 2040 rows and 1680 columns
Presolve time: 0.01s
Presolved: 2043 rows, 2465 columns, 5989 nonzeros
Presolved model has 1120 SOS constraint(s)
Variable types: 0 continuous, 2465 integer (1342 binary)
Found heuristic solution: objective 0.0061728
Found heuristic solutio

[<gurobi.Var x (value 23.0)>,
 <gurobi.Var y (value 29.0)>,
 <gurobi.Var z (value 4.0)>,
 <gurobi.Var single_ind(15*x <= y,) (value -0.0)>,
 <gurobi.Var main_ind(13*x <= 3*y, y < 7*x) (value -0.0)>,
 <gurobi.Var component_ind(13*x <= 3*y,) (value 0.0)>,
 <gurobi.Var component_ind(y < 7*x,) (value 1.0)>,
 <gurobi.Var main_ind(7*x <= y + z, y < 7*x) (value -0.0)>,
 <gurobi.Var component_ind(7*x <= y + z,) (value -0.0)>,
 <gurobi.Var component_ind(y < 7*x,) (value 1.0)>,
 <gurobi.Var single_ind(7*x <= y + z,) (value -0.0)>,
 <gurobi.Var single_ind(7*x <= y - z,) (value -0.0)>,
 <gurobi.Var main_ind(11*x <= 5*y, y < 3*x) (value -0.0)>,
 <gurobi.Var component_ind(11*x <= 5*y,) (value -0.0)>,
 <gurobi.Var component_ind(y < 3*x,) (value 1.0)>,
 <gurobi.Var main_ind(6*x <= 2*y + z, y < 3*x) (value -0.0)>,
 <gurobi.Var component_ind(6*x <= 2*y + z,) (value -0.0)>,
 <gurobi.Var component_ind(y < 3*x,) (value 1.0)>,
 <gurobi.Var main_ind(2*x <= y, y < 3*x, 3*x < y + z) (value -0.0)>,
 <gurobi.Var